In [1]:
import pandas as pd
import numpy as np

# Charger les données
donnees = pd.read_csv("reseau_en_arbre.csv")

# Initialiser la liste des bâtiments pour l'étape une
batiment_etape_une = []
batiment_etape_deux = []

# Obtenez la liste des id_batiment uniques
id_batiments_uniques = donnees['id_batiment'].unique()

# Parcourir chaque id_batiment unique
for id_batiment in id_batiments_uniques:
    # Filtrer les données pour l'id_batiment actuel
    donnees_batiment = donnees[donnees['id_batiment'] == id_batiment]
    
    # Vérifier si toutes les valeurs dans la colonne 'infra_type' sont égales à 'fourreau_existant'
    if all(donnees_batiment['infra_type'] == 'fourreau_existant'):
        # Ajouter l'id_batiment à la liste pour l'étape une
        batiment_etape_une.append(id_batiment)
    else:
        batiment_etape_deux.append(id_batiment)

# Afficher les résultats pour l'étape une
print("Les id_batiment pour l'étape 1 sont :", batiment_etape_une)
#print("Les id_batiment pour lesquels au moins une valeur dans 'infra_type' est différente de 'fourreau_existant' sont :", batiment_etape_deux)


Les id_batiment pour l'étape 1 sont : ['E000001', 'E000002', 'E000004', 'E000005', 'E000006', 'E000007', 'E000008', 'E000009', 'E000010', 'E000011', 'E000012', 'E000013', 'E000014', 'E000015', 'E000016', 'E000017', 'E000018', 'E000019', 'E000020', 'E000021', 'E000022', 'E000023', 'E000024', 'E000025', 'E000026', 'E000027', 'E000028', 'E000029', 'E000030', 'E000031', 'E000032', 'E000033', 'E000034', 'E000035', 'E000036', 'E000037', 'E000038', 'E000039', 'E000040', 'E000041', 'E000042', 'E000043', 'E000044', 'E000045', 'E000046', 'E000047', 'E000048', 'E000049', 'E000050', 'E000051', 'E000052', 'E000053', 'E000054', 'E000055', 'E000056', 'E000057', 'E000058', 'E000059', 'E000060', 'E000061', 'E000062', 'E000063', 'E000064', 'E000065', 'E000066', 'E000068', 'E000069', 'E000070', 'E000071', 'E000072', 'E000073', 'E000074', 'E000075', 'E000076', 'E000077', 'E000078', 'E000079', 'E000080', 'E000081', 'E000082', 'E000083', 'E000084', 'E000086', 'E000087', 'E000088', 'E000089', 'E000090', 'E00

In [2]:
# Calcul de l'indice de difficulté (moyenne longueur par maison)
# Définir la fonction calculer_moyenne_longueur_par_maison
def calculer_moyenne_longueur_par_maison(data, batiment_etape_deux):
    # Convertir les données en un tableau NumPy
    data_array = np.array(data)

    # Initialiser un dictionnaire pour stocker les moyennes
    moyennes = {}

    # Parcourir les id_batiments uniques
    for id_batiment in batiment_etape_deux:
        # Sélectionner les lignes correspondant à l'id_batiment en cours
        lignes_id_batiment = data_array[data_array[:, 0] == id_batiment]
        
        # Récuperer le nombre de prises
        nb_prises = lignes_id_batiment[0, 1]
        
        # Rendre nulles les longueurs associées à infra_type "fourreau_existant"
        lignes_id_batiment[lignes_id_batiment[:, 3] == 'fourreau_existant', 4] = 0
        
        # Calculer la moyenne des longueurs par batiment
        moyenne = np.sum(lignes_id_batiment[:, 4]) / nb_prises
            
        # Stocker la moyenne dans le dictionnaire
        moyennes[id_batiment] = moyenne

    return moyennes

# Initialiser la nouvelle liste ordonnée des batiments
liste_ordonnee_bat_etape_deux = []
infra_ids_mises_a_zero = set()

while batiment_etape_deux:  # Tant que la liste batiment_etape_deux n'est pas vide
    # Appeler la fonction de calcul des moyennes
    moyennes = calculer_moyenne_longueur_par_maison(donnees, batiment_etape_deux)

    # Trouver l'id_batiment avec la plus petite moyenne
    id_batiment_min_moyenne = min(moyennes, key=moyennes.get)
    
    # Récupérer la liste des infra_id associés à cet id_batiment
    infra_ids_min_moyenne = donnees.loc[donnees['id_batiment'] == id_batiment_min_moyenne, 'infra_id'].unique()

    # Récupérer les infra_ids qui ne sont pas déjà à zéro
    infra_ids_a_zero = set(infra_ids_min_moyenne) - infra_ids_mises_a_zero

    # Rendre nulles les longueurs associées à tous les infra_id de l'id_batiment avec la plus petite moyenne (fourreau_existant)
    donnees.loc[(donnees['id_batiment'] == id_batiment_min_moyenne) & (donnees['infra_type'] == 'fourreau_existant'), 'longueur'] = 0

    # Rendre nulles les longueurs associées à tous les infra_id de l'id_batiment avec la plus petite moyenne (fourreau_non_existant)
    donnees.loc[(donnees['id_batiment'] == id_batiment_min_moyenne) & (donnees['infra_type'] == 'fourreau_non_existant'), 'longueur'] = 0

    # Rendre nulles les longueurs associées à tous les infra_id de tous les batiments ayant le même infra_id que l'id_batiment avec la plus petite moyenne
    donnees.loc[(donnees['infra_id'].isin(infra_ids_min_moyenne)) & (donnees['infra_type'].isin(['fourreau_existant', 'fourreau_non_existant'])), 'longueur'] = 0

    # Mettre à jour les infra_ids qui ont été mis à zéro
    infra_ids_mises_a_zero.update(infra_ids_a_zero)
    
    # Supprimer l'id_batiment de la liste initiale
    batiment_etape_deux.remove(id_batiment_min_moyenne)
    
    # Ajouter l'id_batiment à la liste ordonnée
    liste_ordonnee_bat_etape_deux.append(id_batiment_min_moyenne)

    #  # Afficher les infra_ids associés à l'id_batiment avec la plus petite moyenne
    # print(f"Infra_ids associés à l'id_batiment avec la plus petite moyenne ({id_batiment_min_moyenne}):")
    # print(infra_ids_min_moyenne)

# Afficher les résultats finaux
print("Les id_batiment pour l'étape 2 sont : ")
print(liste_ordonnee_bat_etape_deux)


Les id_batiment pour l'étape 2 sont : 
['E000112', 'E000098', 'E000274', 'E000222', 'E000188', 'E000127', 'E000145', 'E000144', 'E000364', 'E000365', 'E000111', 'E000185', 'E000267', 'E000268', 'E000067', 'E000252', 'E000301', 'E000315', 'E000316', 'E000128', 'E000373', 'E000376', 'E000374', 'E000377', 'E000380', 'E000378', 'E000381', 'E000266', 'E000293', 'E000085', 'E000106', 'E000107', 'E000183', 'E000322', 'E000346', 'E000359', 'E000366', 'E000108', 'E000109', 'E000110', 'E000325', 'E000326', 'E000184', 'E000129', 'E000360', 'E000367', 'E000368', 'E000361', 'E000369', 'E000372', 'E000375', 'E000379', 'E000003', 'E000146', 'E000194', 'E000195', 'E000196', 'E000225', 'E000258', 'E000259', 'E000302', 'E000257', 'E000178', 'E000197', 'E000226', 'E000227', 'E000228', 'E000229', 'E000198', 'E000260', 'E000275', 'E000317', 'E000327', 'E000358', 'E000363', 'E000370', 'E000303', 'E000371', 'E000160', 'E000161', 'E000176', 'E000177', 'E000174', 'E000175', 'E000193']
